In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [2]:
key_path = key_path
project = project_id

# Sources
table = 'gold_main_sp500'
dataset = 'gold'

# Tables id
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'silver_pca_sector_industry'
dataset_to_save = 'silver'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
df = bigquery.run_query(
    f"""
    SELECT
        *
    FROM {project}.{dataset}.{table}
    """
)
df

,date,open,high,low,close,adjclose,volume,ticker,id,SMA_20,...,industry_Tobacco,industry_Tools___Accessories,industry_Travel_Services,industry_Trucking,industry_Utilities_Diversified,industry_Utilities_Independent_Power_Producers,industry_Utilities_Regulated_Electric,industry_Utilities_Regulated_Gas,industry_Utilities_Regulated_Water,industry_Waste_Management
0,2018-07-06 00:00:00+00:00,2077.000000,2089.610107,2063.419922,2086.929932,2086.929932,252800,BKNG,006b6e99c62910d9caeb30bf3db010f6,1925.863000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-07-25 00:00:00+00:00,150.479996,151.580002,148.750000,151.360001,142.402573,513000,LHX,615e830346c346cd496c7fae11acab1f,145.932499,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-07-02 00:00:00+00:00,112.870003,114.709999,112.690002,114.529999,114.529999,832900,DECK,7c5f219bc6036984a41259c002072acb,79.722000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-07-12 00:00:00+00:00,90.068085,90.340424,89.574471,90.144684,80.291634,1144450,DTE,7469bcb95008daf6378cd515d18635d6,90.281277,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2018-07-12 00:00:00+00:00,67.120003,68.309998,66.730003,68.070000,65.105553,1287100,XYL,7963531541a59d24df0c7d83299e088e,66.346000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183474,2015-10-06 00:00:00+00:00,29.610001,29.820000,29.530001,29.715000,25.431040,3700800,AFL,d01d31825eb895e2ef481bb7f54babc6,55.913000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1183475,2019-04-12 00:00:00+00:00,80.150002,80.739998,80.010002,80.519997,76.745789,909000,AJG,1336ad317993918b26656b93673d6f43,72.317500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1183476,2019-04-01 00:00:00+00:00,43.349998,44.240002,43.200001,44.130001,42.266273,5281300,SCHW,249984f05a9ddb2e4bc1a14c3e80116d,48.247000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1183477,2019-04-25 00:00:00+00:00,175.509995,178.229996,175.179993,178.009995,167.045578,1210100,CME,3f0d86da444c93c35a6c4bb31493fb4d,178.103000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado df con las variables que quieres agrupar
# Primero, estandarizamos los datos

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Aplicar PCA con todas las componentes
pca = PCA()
pca_result = pca.fit_transform(df_scaled)

# Ver la cantidad de varianza explicada por cada componente
print(pca.explained_variance_ratio_)

ValueError: could not convert string to float: 'BKNG'

In [ ]:
# Graficar la varianza explicada por cada componente
plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o', linestyle='--')
plt.title('Cumulative Explained Variance by PCA')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid()
plt.show()

In [ ]:
# Loadings: cómo cada variable original contribuye a los componentes principales
loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i}' for i in range(1, len(df.columns)+1)], index=df.columns)
print(loadings)

In [ ]:
# Graficar las primeras dos componentes principales
plt.figure(figsize=(8, 5))
plt.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.7)
plt.title('PCA - Plot of First Two Principal Components')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True)
plt.show()

In [ ]:
# Convertimos las variables categoricas en dummies
# columns_to_encode = ['sector', 'industry']
# categorical_df = pd.DataFrame([], columns=[])
# 
# # Aplicamos el one hot encoding
# for col in columns_to_encode:
#   new_cols_oneHot = pd.get_dummies(df[col], prefix=col)*1.0 # multiplico por 1.0 para convertir True/False a números directamente
#   categorical_df = pd.concat([categorical_df, new_cols_oneHot], axis=1)
# 
# # Unimos en el mismo df
# df_final = pd.concat([df, categorical_df], axis=1, join='inner')
# 
# # Eliminamos las variables categoricas que ya hemos transformado
# df_final = df_final.drop(columns=columns_to_encode)
# 
# df_final.info()


In [5]:
# Pivotar el DataFrame
df_pivot = macro_data.pivot(index=['date'], columns='serie_id', values='value')

# Resetear el índice para que 'date' sea una columna en lugar de un índice
df_pivot = df_pivot.reset_index()

# Normalizamos la fecha
df_pivot['date'] = df_pivot['date'].dt.tz_localize(None)


df_pivot

serie_id,date,BOPGSTB,CPIAUCSL,FEDFUNDS,GDP,PPIACO,RSAFS,UMCSENT,UNRATE
0,2015-01-01,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7
1,2015-02-01,-33903.0,235.342,0.11,NaN,191.100,435450.0,95.4,5.5
2,2015-03-01,-48732.0,235.976,0.11,NaN,191.500,441895.0,93.0,5.4
3,2015-04-01,-39836.0,236.222,0.12,18279.784,190.900,442967.0,95.9,5.4
4,2015-05-01,-38502.0,237.001,0.12,NaN,193.400,446114.0,90.7,5.6
...,...,...,...,...,...,...,...,...,...
110,2024-03-01,-68582.0,312.230,5.33,NaN,255.095,703738.0,79.4,3.8
111,2024-04-01,-74462.0,313.207,5.33,28652.337,256.984,702681.0,77.2,3.9
112,2024-05-01,-75006.0,313.225,5.33,NaN,255.268,704309.0,69.1,4.0
113,2024-06-01,-73109.0,313.049,5.33,NaN,255.779,702862.0,68.2,4.1


In [6]:
# Convertimos 'date' a tipo datetime
macro_data['date'] = pd.to_datetime(macro_data['date'])

# Forzamos que sea sin zona horaria
macro_data['date'] = macro_data['date'].dt.tz_localize(None)

# Obtiene la fecha mínima
min_date = macro_data['date'].min()

# Obtiene la fecha de hoy como naive
end_date = pd.Timestamp.today().normalize()  # normalize() para eliminar la hora

# Generar una serie de fechas desde la fecha mínima hasta hoy
date_range = pd.date_range(start=min_date, end=end_date)

# Crear un nuevo DataFrame con la columna 'date'
df_dates = pd.DataFrame(date_range, columns=['date'])

df_dates

,date
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-04
4,2015-01-05
...,...
3525,2024-08-26
3526,2024-08-27
3527,2024-08-28
3528,2024-08-29


In [7]:
df = pd.merge(df_dates, df_pivot, on=['date'], how='left')

# Resetear el índice para que 'date' sea una columna en lugar de un índice
df = df.reset_index(drop=True)

df.columns

Index(['date', 'BOPGSTB', 'CPIAUCSL', 'FEDFUNDS', 'GDP', 'PPIACO', 'RSAFS',
       'UMCSENT', 'UNRATE'],
      dtype='object')

In [8]:
# Asegúrate de que la columna 'date' esté en formato datetime y ordena el DataFrame
df = df.sort_values(by='date')

# Rellenar los valores nulos con el valor anterior
df['BOPGSTB'] = df['BOPGSTB'].fillna(method='ffill')
df['CPIAUCSL'] = df['CPIAUCSL'].fillna(method='ffill')
df['FEDFUNDS'] = df['FEDFUNDS'].fillna(method='ffill')
df['GDP'] = df['GDP'].fillna(method='ffill')
df['PPIACO'] = df['PPIACO'].fillna(method='ffill')
df['RSAFS'] = df['RSAFS'].fillna(method='ffill')
df['UMCSENT'] = df['UMCSENT'].fillna(method='ffill')
df['UNRATE'] = df['UNRATE'].fillna(method='ffill')

df

,date,BOPGSTB,CPIAUCSL,FEDFUNDS,GDP,PPIACO,RSAFS,UMCSENT,UNRATE
0,2015-01-01,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7
1,2015-01-02,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7
2,2015-01-03,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7
3,2015-01-04,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7
4,2015-01-05,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7
...,...,...,...,...,...,...,...,...,...
3525,2024-08-26,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3
3526,2024-08-27,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3
3527,2024-08-28,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3
3528,2024-08-29,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3530 entries, 0 to 3529
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      3530 non-null   datetime64[ns]
 1   BOPGSTB   3530 non-null   float64       
 2   CPIAUCSL  3530 non-null   float64       
 3   FEDFUNDS  3530 non-null   float64       
 4   GDP       3530 non-null   float64       
 5   PPIACO    3530 non-null   float64       
 6   RSAFS     3530 non-null   float64       
 7   UMCSENT   3530 non-null   float64       
 8   UNRATE    3530 non-null   float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 248.3 KB


In [10]:
# Fields that make up the ID
id_fields = ['date']

# Apply the function to the DataFrame to create the 'id' column
df['id'] = df.apply(generate_id, axis=1, fields=id_fields)

df

,date,BOPGSTB,CPIAUCSL,FEDFUNDS,GDP,PPIACO,RSAFS,UMCSENT,UNRATE,id
0,2015-01-01,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,4c6f3a3ad2c3295df78d540b586fa7a3
1,2015-01-02,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,d046a975255510ffcbbf9698294fdf40
2,2015-01-03,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,839cd1c08a392d57d500f00285268f2d
3,2015-01-04,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,0518f2b63236a9bc3119f02cabae4fd4
4,2015-01-05,-38865.0,234.747,0.11,18063.529,192.000,436125.0,98.1,5.7,71e7b2696f675336f79208247cdc5594
...,...,...,...,...,...,...,...,...,...,...
3525,2024-08-26,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,ef84b4618ba298b6579599a9133cb02c
3526,2024-08-27,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,d03c334c76ec10668a8689d5490c227f
3527,2024-08-28,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,c03f67e2119c230ed5198baf394982cf
3528,2024-08-29,-73109.0,313.534,5.33,28652.337,257.723,709668.0,68.2,4.3,a2125ebc5a8ad74a15dbf2b516e983c5


In [21]:
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='id', table=table_conca_to_save, new_df=df)

    if not df_incremental.empty:
        # Guardamos los datos en Bigquery
        bigquery.save_dataframe(df_incremental, project, dataset_to_save, table_to_save, if_exists='append', schema=None)
        print(f'New records loaded')
    else:
        print('No new records to load.')

# En el caso de no tener datos en Bigquery, guardamos todo el df
except Exception as e:
    bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)
    print('New data persisted')
    print(f'Exception encountered: {e}')

No new records to load.
